### composite analysis for thermodynamic variabilities

1. create dataset <br>
2. [read data](#readdata)

In [1]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [2]:
warnings.filterwarnings('ignore')

In [3]:
#ISCCP_dir = '/w2-data2/willytsai/ISCCP/hgg/'
TRMM_dir = '/data2/willytsai/TRMM_3hr/TRMM/2009_2018'
#MERRA2_dir = '/w2-data2/willytsai/MERRA2/'
#MERRA2_3d_dir = '/w2-data/willytsai/'
#ERA5_dir = '/w2-data/willytsai/ERA5/'
#Gridsat_dir = '/w2-data2/willytsai/gridsat_BT/remap_0.25deg/'

In [4]:
# %%time
# # MERRA2 dataset 2014-2015, hourly in a single file
# os.chdir(MERRA2_dir+'inst_2d_hourly/')
# files = sorted(glob('MERRA2_400.inst1_2d_asm_Nx.201*'))
# data = xr.open_mfdataset(files)
# data = data.resample(time='3H').nearest() # CWV
# cwv_xr = data.TQV

In [5]:
#%%time 
#os.chdir(ERA5_dir)
#files = sorted(glob('*.nc'))
#data = xr.open_mfdataset(files)
#cwv_era_xr = data.tcwv

In [6]:
date = []; date_begin = datetime(2014,1,1)
for t in range(14608):
    tmp = date_begin + t*timedelta(hours=3)
    date.append(tmp)

In [7]:
%%time
# TRMM3B42 dataset 2014, 3hrly separate
yr = ['2009_2018']
prec_3hr = np.nan*np.zeros((len(date),241,1440))

n=1
os.chdir(TRMM_dir)
files = sorted(glob('*2014*')+glob('*2015*')+glob('*2016*')+glob('*2017*')+glob('*2018*'))
for file in files:
    data = xr.open_dataset(file).precipitation        
    prec_3hr[n-1,:,:] = data.values.swapaxes(0,1)
    n+=1
lon = data.nlon; lat = data.nlat
prec_xr = xr.DataArray(prec_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del prec_3hr

CPU times: user 2min 56s, sys: 1min 9s, total: 4min 5s
Wall time: 6min 3s


In [8]:
# %%time
# # MERRA2 reanalysis 2014, 3hrly 

# os.chdir(MERRA2_3d_dir+'TQUV_3hr/')
# files = sorted(glob('MERRA2_400.inst3_3d_asm_Np.2014*')
#                + glob('MERRA2_400.inst3_3d_asm_Np.2015*'))
# data = xr.open_mfdataset(files)
# T_xr = data.T   
# q_xr = data.QV  
# u_xr = data.U
# v_xr = data.V

# # wind shear
# llwsz_xr = np.sqrt((u_xr.sel(lev=900)-u_xr.sel(lev=700))**2 + 
#                   (v_xr.sel(lev=900)-v_xr.sel(lev=700))**2)

# os.chdir(MERRA2_3d_dir+'omega_3hr/')
# files = sorted(glob('MERRA2_400.inst3_3d_asm_Np.2014*')
#                + glob('MERRA2_400.inst3_3d_asm_Np.2015*'))
# data = xr.open_mfdataset(files)
# Omega_xr = data.OMEGA

In [9]:
# %%time
# # MERRA2 OLR dataset 2014-2015, hourly in a single file
# os.chdir(MERRA2_dir+'inst2d_OLR')
# files = sorted(glob('MERRA2_400.tavg1_2d_rad_Nx.2014*')
#               + glob('MERRA2_400.tavg1_2d_rad_Nx.2015*'))
# data = xr.open_mfdataset(files)
# data = data.resample(time='3H').nearest() # CWV
# olr_xr = data.LWTUP

In [10]:
# %%time
# # Gridsat-BT dataset 2014, 2015 3hourly
# yr = ['2014','2015']
# bt_3hr = np.nan*np.zeros((len(date),241,1440))

# n=1
# for y in yr:
#     os.chdir(Gridsat_dir+y)
#     files = sorted(glob('GRIDSAT-B1.*'))
#     for file in files:
#         data = xr.open_dataset(file)
#         bt_3hr[n-1,:,:] = data.irwin_cdr
#         n+=1
# lon = data.lon; lat = data.lat
# bt_xr = xr.DataArray(bt_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del bt_3hr

In [11]:
# # derive Relative hunidity
# lev = T_xr.lev
# es = 6.1094*np.exp(17.625*(T_xr-273)/(T_xr-273+243.04))
# p = (q_xr/q_xr)*lev
# qs = es/(p-es)*0.622
# rh_xr = q_xr/qs*100 # relative humidity [%] 

In [12]:
# %%time
# cldamt_type_3hr = np.nan*np.zeros((len(date),18,180,360))
# cldamt_3hr = np.nan*np.zeros((len(date),180,360))
# # ISCCP dataset 2014, 3hourly in a single file
# yr = ['2014','2015'] # year
# mn = ['01','02','03','04','05','06','07','08',
#      '09','10','11','12'] # month

# n=1
# for y in yr:
#     for m in mn:
#         os.chdir(ISCCP_dir+y+m)
#         files = sorted(glob('*.nc'))
        
#         for file in files:
#             data = xr.open_dataset(file).cldamt_types # cloud amount [%]
#             data2 = xr.open_dataset(file).cldamt
#             tmp = data; tmp2 = data2
#             cldamt_type_3hr[n-1,:,:,:] = tmp
#             cldamt_3hr[n-1,:,:] = tmp2
#             n+=1
# lat = tmp.lat; lon = tmp.lon
# print(n-1)
# cldamt_type_xr = xr.DataArray(cldamt_type_3hr,coords=[date,range(18),lat,lon],dims=['time','type','lat','lon']); del cldamt_type_3hr
# cldamt_xr = xr.DataArray(cldamt_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del cldamt_3hr

In [13]:
# lat = cldamt_type_xr.lat
# lon = cldamt_type_xr.lon
# # 0+3 shallow cumulus, 2+5 stratoform, 14+17 deep convection 
# cldamt_cu = (cldamt_type_xr[:,0,:,:]+cldamt_type_xr[:,3,:,:]).values
# cldamt_cu[cldamt_cu>100] = np.nan; cldamt_cu[cldamt_cu< 0] = np.nan;
# cu_xr = xr.DataArray(cldamt_cu,coords=[date,lat,lon],dims=['time','lat','lon']); del cldamt_cu

# cldamt_st = (cldamt_type_xr[:,13,:,:]+cldamt_type_xr[:,16,:,:]).values
# cldamt_st[cldamt_st>100] = np.nan; cldamt_st[cldamt_st< 0] = np.nan;
# st_xr = xr.DataArray(cldamt_st,coords=[date,lat,lon],dims=['time','lat','lon']); del cldamt_st

# cldamt_dc = (cldamt_type_xr[:,14,:,:]+cldamt_type_xr[:,17,:,:]).values
# cldamt_dc[cldamt_dc>100] = np.nan; cldamt_dc[cldamt_dc< 0] = np.nan;
# dc_xr = xr.DataArray(cldamt_dc,coords=[date,lat,lon],dims=['time','lat','lon']); del cldamt_dc

### functions

In [14]:
def fft_filter(data,filtered_day,span_days):
    
    Vp = (data - data.mean()) # olr anomaly
    Vspec = np.fft.fft(Vp) 

    NT=len(data)
    freqs = np.array(range(NT))   # This gets the left end correct
    reversed_freqs = freqs[::-1]  # Reverse the array 
    reversed_freqs[0:int(NT/2)] = freqs[1:int(NT/2)+1] # Fix the low end of the reversed array
    freqs = reversed_freqs[::-1]  # reverse it back 
    
    # filtering and reconstructing data
    Vspec_filt = Vspec.copy()
    Vspec_filt[np.where(span_days/freqs < filtered_day)] = 0 # 60/freqs [day], filter out periods less than 2 day 
    Vp_recon = np.fft.ifft(Vspec_filt).real
    
    return Vp_recon + data.mean()

In [15]:
def readdata(lat_s,lat_n,lon_w,lon_e,start_date,end_date):
    
    global prec_xr#,cwv_era_xr,cwv_xr,dc_xr,cu_xr,st_xr,cldamt_xr
#    global T_xr,q_xr,u_xr,v_xr,Omega_xr,rh_xr,olr_xr,bt_xr
    
    prec_sm = prec_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
                      time=slice(start_date,end_date))
#     cwv_sm = cwv_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     cwv_era_sm = cwv_era_xr.sel(latitude=slice(lat_n,lat_s),longitude=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     dc_sm = dc_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     cu_sm = cu_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     st_sm = st_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     cldamt_sm = cldamt_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
    
#     T_sm = T_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     q_sm = q_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     u_sm = u_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     v_sm = v_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     omega_sm = Omega_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     rh_sm = rh_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     olr_sm = olr_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     bt_sm = bt_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))
#     llwsz_sm = llwsz_xr.sel(lat=slice(lat_s,lat_n),lon=slice(lon_w,lon_e),
#                       time=slice(start_date,end_date))

    return prec_sm

In [16]:
def org_index(date,prec_sm,olr_sm,bt_sm):
    
    N_ps = np.zeros(prec_sm.shape[0]) # OLR_based
    N_psT = np.zeros(prec_sm.shape[0]) # BT_based
    N_prec = np.zeros(prec_sm.shape[0]) # prec_based
    cldfrac = np.zeros(prec_sm.shape[0]) # cldfrac over domain by OLR criteria
    precfrac = np.zeros(prec_sm.shape[0]) # cldfrac over domain by OLR criteria
    SCAI_BT = np.zeros(prec_sm.shape[0]) # cldfrac over domain by OLR criteria
    SCAI_p = np.zeros(prec_sm.shape[0]) # cldfrac over domain by OLR criteria

    
    # N and SCAI indices
    for t in range(prec_sm.shape[0]):
        label_obj, N = scai.labeled_obj(olr_sm[t,:,:].values,cri=180,flag=2) # less than 220 W/m^2
        N_ps[t] = N

    for t in range(prec_sm.shape[0]):
        label_obj, N = scai.labeled_obj(bt_sm[t,:,:].values,cri=240,flag=2) # less than 220 W/m^2
        SCAI_BT[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)
        N_psT[t] = N
    
    for t in range(prec_sm.shape[0]):
        label_obj, N = scai.labeled_obj(prec_sm[t,:,:].values,cri=1,flag=1) # larger than 1mm/day
        SCAI_p[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)    
        N_prec[t] = N

    N_ps = xr.DataArray(N_ps,coords=[date],dims=['time'])
    N_psT = xr.DataArray(N_psT,coords=[date],dims=['time'])
    N_prec = xr.DataArray(N_prec,coords=[date],dims=['time'])
    
    
    # cloud fraction (by OLR < 180)
    for t in range(prec_sm.shape[0]):
        tmp = len(np.where(bt_sm[t,:,:].values<240)[0])/(bt_sm[t,:,:].shape[0]*bt_sm[t,:,:].shape[1])
        cldfrac[t] = tmp
    cldfrac = xr.DataArray(cldfrac,coords=[date],dims=['time'])
    
    # prec fraction (by prec > 1mm/hr)
    for t in range(prec_sm.shape[0]):
        tmp = len(np.where(prec_sm[t,:,:].values>1)[0])/(prec_sm[t,:,:].shape[0]*prec_sm[t,:,:].shape[1])
        precfrac[t] = tmp
    precfrac = xr.DataArray(precfrac,coords=[date],dims=['time'])
    
    return (N_ps, N_psT, N_prec, SCAI_BT, SCAI_p, cldfrac, precfrac)

In [17]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N
    y_avg = np.convolve(y, avg_mask, 'same')
    return y_avg

### Composite time series for thermodynamic variability based on degree of organization

In [18]:
### number of precipitating systems 
os.chdir('/data2/willytsai/python_module')
import SCAI_calc4obj as scai

#### 1. Domain and Time 

In [19]:
# time window
start_date = datetime(2014,1,1)
end_date = datetime(2019,1,1)

# INDO = lat(-10,10), lon(65,75)
lat_s,lat_n = -10,10
lon_w,lon_e = 50,90
#lon_w,lon_e = 60,70
deg_box = 10 
overlap = 2.5

lat_bnd = np.linspace(lat_s,lat_n,int((lat_n-lat_s)/overlap)+1) 
lon_bnd = np.linspace(lon_w,lon_e,int((lon_e-lon_w)/overlap)+1)
lat_grid = lat_bnd[1:-1]
lon_grid = lon_bnd[1:-1]

# W_Pacific = lat(-10,10), lon(150,180)
lon_w,lon_e = 150,180

lon_bnd = np.linspace(lon_w,lon_e,int((lon_e-lon_w)/overlap)+1)
lon_grid = np.concatenate((lon_grid,lon_bnd[1:-1]))

# E_Pacific = lat(-10,10), lon(150,180)
lon_w,lon_e = -180,-90

lon_bnd = np.linspace(lon_w,lon_e,int((lon_e-lon_w)/overlap)+1)
lon_grid = np.concatenate((lon_grid,lon_bnd[1:-1]))
print(lat_grid)
print(lon_grid)

[-7.5 -5.  -2.5  0.   2.5  5.   7.5]
[  52.5   55.    57.5   60.    62.5   65.    67.5   70.    72.5   75.
   77.5   80.    82.5   85.    87.5  152.5  155.   157.5  160.   162.5
  165.   167.5  170.   172.5  175.   177.5 -177.5 -175.  -172.5 -170.
 -167.5 -165.  -162.5 -160.  -157.5 -155.  -152.5 -150.  -147.5 -145.
 -142.5 -140.  -137.5 -135.  -132.5 -130.  -127.5 -125.  -122.5 -120.
 -117.5 -115.  -112.5 -110.  -107.5 -105.  -102.5 -100.   -97.5  -95.
  -92.5]


#### Main frame

In [20]:
var_table = ['prec_sm', 'cwv_sm', 'cwv_era_sm', 'dc_sm', 'cu_sm', 'st_sm', 'cldamt_sm'
           ,'T_sm', 'q_sm', 'u_sm', 'v_sm', 'omega_sm', 'rh_sm', 'olr_sm', 'bt_sm', 'llwsz_sm']

In [21]:
%%time 

var_list = [] # total events 

for lat in lat_grid:
    for lon in lon_grid:
        metadata = readdata(lat-deg_box/2,lat+deg_box/2,lon-deg_box/2,lon+deg_box/2,
                    start_date,end_date)
        #(prec_sm, cwv_sm, cwv_era_sm, dc_sm, cu_sm, st_sm, cldamt_sm, T_sm,
        #    q_sm, u_sm, v_sm, omega_sm, rh_sm, olr_sm, bt_sm, llwsz_sm) = metadata  
        
        prec_sm = metadata
        
        #(N_ps, N_psT, N_prec, SCAI_BT, SCAI_p, cldfrac, precfrac) = org_index(date,prec_sm,olr_sm,bt_sm)
        # calculate domain-average variables for 2D
        prec_dmean = (prec_sm.mean(axis=1)).mean(axis=1).values         
        cwv_dmean = 0 #(cwv_sm.mean(axis=1)).mean(axis=1).values         
        dc_dmean = 0 #(dc_sm.mean(axis=1)).mean(axis=1).values         
        cu_dmean = 0 #(cu_sm.mean(axis=1)).mean(axis=1).values         
        st_dmean = 0 #(st_sm.mean(axis=1)).mean(axis=1).values         
        cldamt_dmean = 0 #(cldamt_sm.mean(axis=1)).mean(axis=1).values     
        olr_dmean = 0 #(olr_sm.mean(axis=1)).mean(axis=1).values  
        bt_dmean = 0 #(bt_sm.mean(axis=1)).mean(axis=1).values  
        llwsz_dmean = 0 #(llwsz_sm.mean(axis=1)).mean(axis=1).values  
                
        # July 20, 2020 1-day running mean 
        #prec_dmean = running_mean(prec_dmean,8)
        
        # define convective events by rainfall
        t_event = []

        t_all = np.where(prec_dmean*24>5)[0] # averaged rainfall > 5 mm/day
        t_all = t_all[np.logical_and(t_all > 16,t_all < len(date)-17)] 
        for t in t_all:
            p_window = prec_dmean[t-16:t+17]
            if p_window.max() == p_window[16]: # prec max within 3d
                t_event.append(t)        
        t_event = np.asarray(t_event)
        
        # make sample list containing variables of subdomains        
        for t in t_event:
            tmp = [date[t],lat,lon,prec_dmean[t]*24,0,0, \
                           0,0,0 \
                           ,0,0,0, \
                           0,0,0 \
                           ,0,0,0,0, \
                            0]
            var_list.append(tmp)   

# convert list to array            
var_list = np.asarray(var_list)

CPU times: user 1min 28s, sys: 52.5 s, total: 2min 20s
Wall time: 2min 20s


In [22]:
var_name = ['date','lat','lon','prec_dmean','cwv_dmean_MERRA2','cwv_dmean_ERA',
           'dc_cldamt_dmean','cu_cldamt_dmean','st_cldamt_dmean','cldamt_dmean',
           'olr_dmean','bt_dmean','N_olr','N_BT','N_prec','SCAI_BT','SCAI_p',
           'cldfrac_BT','precfrac','llws_dmean'] 

In [23]:
# save variable list
np.savetxt('/data/willytsai/ISCCP/var_name_2014_2018_10deg_4ds.dat',var_name,fmt='%16s')

In [24]:
# save time 
time_str = np.zeros(var_list.shape[0],dtype='U13')
for n in range(var_list.shape[0]):
    time_str[n] = var_list[n,0].strftime('%Y-%m-%d-%H')
    
np.savetxt('/data/willytsai/ISCCP/time_event_2014_2018_10deg_4ds.dat',time_str,fmt='%13s')

In [25]:
# save dataset
np.savetxt('/data/willytsai/ISCCP/var_event_2014_2018_10deg_4ds.dat',var_list[:,1:])